export cells as modules?



In [ ]:
var fs = require('fs');
var path = require('path');
var rimraf = require('rimraf');
var importer = require('../Core');
var mkdirpSync = importer.import('mkdirp');
var getExtension = importer.import('cell extension')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var MODULE_OUTPUT = __dirname + '/.modules';
if(fs.existsSync(MODULE_OUTPUT)) {
    rimraf.sync(MODULE_OUTPUT);
}

function exportNotebook(notebook, project) {
    if(!project) {
        project = __dirname + '/..';
    }
    const notebooks = typeof notebook === 'string' ? [notebook] : notebook;
    return notebooks.reduce((cells, notebook) => {
        const name = path.basename(notebook).replace(/\.ipynb/ig, '');
        const parent = path.resolve(path.dirname(notebook)).replace(path.resolve(project), '');
        mkdirpSync(path.join(MODULE_OUTPUT, parent, name));

        return importer.getCells(notebook, ['*', 'code', 'markdown', 'raw'])
            .reduce((results, cell, i) => {
                const extension = getExtension(cell, notebook);
                // TODO: get previous markdown and extract name that leads back to the current cell
                const cellPath = path.join(MODULE_OUTPUT, parent, name, 'cell-' + i + extension);
                fs.writeFileSync(cellPath, cell.source.join(''));
                results.push(cellPath);
                return results;
            }, cells);
    }, []);
}
module.exports = exportNotebook;


get cell extension?


In [ ]:

function getExtension(cell, notebook) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: ' + (cell.filename || notebook) + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}
module.exports = getExtension;


export all notebooks?

Export all notebooks to a structured folder where each cell has it's own file for linting?



In [ ]:
var glob = require('glob');
var path = require('path');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

function exportAll(project) {
    const notebooks = glob.sync('**/*.ipynb', {cwd: project});
    return exportNotebook(notebooks.map(n => path.join(project, n)), project);
}
module.exports = exportAll;


test notebook export?



In [ ]:
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var fs = require('fs');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = path.resolve(__dirname + '/../');
var MODULE_OUTPUT = PROJECT_PATH + '/Utilities/.modules';

var sortNumeric = (that) => {
    return that.sort((a, b) => parseInt(a.split(/[^0-9]/ig).join('')) - parseInt(b.split(/[^0-9]/ig).join('')));
}

describe('notebook export service', () => {
    it('should export this file', () => {
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        assert(path.basename(files[0]) === 'cell-0.md');
        assert(path.basename(files[1]) === 'cell-1.js');
        assert(path.basename(files[2]) === 'cell-2.md');
    })
    
    it('should have comparable cells', () => {
        var importCells = importer.interpret(['notebook.ipynb[0]', 'notebook.ipynb[1]', 'notebook.ipynb[2]']);
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        var fsCell0 = fs.readFileSync(path.join(MODULE_OUTPUT, files[0])).toString();
        var fsCell1 = fs.readFileSync(path.join(MODULE_OUTPUT, files[1])).toString();
        var fsCell2 = fs.readFileSync(path.join(MODULE_OUTPUT, files[2])).toString();
        assert(fsCell0 === importCells[0].markdown.join(''));
        assert(fsCell1 === importCells[0].fresher, fsCell1.length + ' != ' + importCells[0].fresher.length)
        assert(fsCell2 === importCells[1].markdown.join(''))
    })
    
    it('should export all notebooks', () => {
    //    var exported = exportAll(PROJECT_PATH);
    //    assert(exported.length > 0);
    })
})


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


In [ ]:
// TODO: delete .modules if everything checks out

// TODO: import tests


TODO:

import module as notebook cell?

import gist as notebook

import random instructions as notebook

A couple of rules:
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable in isolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'


```javascript

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}

```
- every module should have a markdown title, at least one question that the code intends to answer (how to?), includes at least (where name is the name of any function in the code block):


```javascript

...
function <name> () {
...
}
...
exports = <name>
...

```

 - eliminate circular dependencies the same way unintended recursion is avoided, create a condition:
 
ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
var funcA = require('ModuleA.js');
```

Becomes:

ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
if(typeof funcA === 'undefined') {
    var funcA = require('ModuleA.js');
}
```

 - parameters are listed in most specific left to least specific right, i.e. function(filter, context) would mean filter is only used for this function, whereas context may be passed in to this function as well as other functions.  "filter" is on the left because it is specifically used just for this function, context is on the right because it might contain a path on the filesystem, or some options.
 - Entry cells, i.e. cells that are intended to be called by a service, coordinate interactions between multiple services - should not contain catch blocks so that the task scheduler will fail and log the last error.




get imports recursively?

TODO: use this in file-system graph

TODO: AST compatible map of import functions across all languages
Python `__import__`, Java: `import`, C#: `using`, C++: `#import`.  Basic functionality of JetBrains intellisense.



In [ ]:
var importer = require('../Core');
var importerCode = importer.interpret('import notebook.ipynb');
var importerReg = /(interpret|runAllPromises|runInNewContext|getCells|streamJson|regexToArray)/igm;
var importsReg = /(importNotebook|\.import)\(((\s*([\['"\{])\s*([\s\S]*?)\s*\3[,\s]*)+)\)/ig;

// TODO: graduate this to notebook interpreter for use in other analysis
function matchImports(code) {
    var imports = importer.regexToArray(importsReg, code, 2);
    var coreMatches = importer.regexToArray(importerReg, code, 1)
        .map(i => importerCode.filter(core => core.code.match(new RegExp('exports.*?' + i, 'ig')))[0].id);
    return imports.map(i => {
        const importStrings = importer.regexToArray(/([\['"\{])\s*([\s\S]*?)\s*\1/ig, i, 2);
        if(i.includes('[')) {
            return importStrings;
        }
        return importStrings[0];
    }).concat(coreMatches);
}

function getImportsRecursively(interpretedImports, results, searches) {
    var imports = {};
    searches.forEach((search, i) => {
        var result = results[i];
        if(typeof interpretedImports[search] !== 'undefined') {
            return;
        }
        interpretedImports[search] = result;
        if(!result.length) {
            result = [result];
        }
        result.forEach(cell => {
            interpretedImports[cell.id] = cell;
            imports[cell.id] = matchImports(cell.code);
            imports[cell.id].forEach(arr => {
                // normalize input incase importing an array like .import(['search'])
                if(typeof arr === 'string') arr = [arr];
                var result = importer.interpret(arr);
                Object.assign(imports, getImportsRecursively(interpretedImports, result, arr));
            });
        })
    });
    
    return imports;
}

module.exports = getImportsRecursively;



rename cell to a nice name?

In [ ]:
var importer = require('../Core');
var getExtension = importer.import('cell extension')

// get previous markdown and extract name that leads back to the current cell
function niceName(cell) {
    cell.questions.sort((a, b) => a.length - b.length);
    return cell.questions[0].replace('?', '').replace(/[^a-z0-9]+|\btest\b/igm, ' ').trim().replace(/\s+/igm, '-') + getExtension(cell);
}

module.exports = niceName;


add previously imported code?

In [ ]:
var _ = require('lodash');
var importer = require('../Core');
var niceName = importer.import('rename cell nice name');

var REGEX_EXPORTS = /exports[^=]*?\s*=\s*([^\s;]*)/igm;
var REGEX_NAMED = /function\s+([^\( ]*)\s*\(/igm

function getExports(code) {
    const named = importer.regexToArray(REGEX_NAMED, code, 1);
    const namedExports = importer.regexToArray(REGEX_EXPORTS, code, 1);
    return _.intersection(named, namedExports);
}

function addImports(exportedCode, imports) {
    const newExports = getExports(exportedCode);
    // include all functional references from cells above this current cell
    Object.keys(imports).forEach(k => {
        const includedCell = imports[k];
        const funcName = getExports(includedCell.code)[0];
        if(funcName
            && !newExports.includes(funcName)
            && exportedCode.match(new RegExp(funcName + '\\s*\\(', 'g'))
            && !exportedCode.match(new RegExp('(let|var)\\s+' + funcName + '|(let|var)\\s+\\{[^\\}]*?' + funcName, 'ig'))
            && includedCell.questions.length > 0) {
            // TODO: replace this with a polyfill across ALL notebooks
            exportedCode = `
var ${funcName} = (() => {
    const r = require('./${niceName(includedCell)}');
    return typeof r === 'function' ?
        r
        : (r && typeof r[Object.keys(r)[0]] === 'function'
            ? r[Object.keys(r)[0]]
            : r);
})();
${exportedCode}`;
        }
    });
    return exportedCode;
}

module.exports = addImports;


replace notebook imports?

automatic jupyter dependency injector?

TODO: Use and AST instead of regexp



In [ ]:
var importer = require('../Core');
var addImports = importer.import('add previously imported code');
var niceName = importer.import('rename cell nice name');

var importerReg = /importer\.(interpret|runAllPromises|runInNewContext|getCells|streamJson|regexToArray)/igm;

// How to escape a string for regex?
function escapeRegExp(str) {
    return str.replace(/[\-\[\]\/\{\}\(\)\*\+\?\.\\\^\$\|]/g, '\\$&');
}

// TODO: use an AST for all of this static analysis
function replaceImports(exports, cell, allImports) {
    cell.name = niceName(cell);
    // TODO: use AST for this, doesn't work in quotes
    if(cell.filename.includes('get all cached')) {
        return cell.code;
    }
    var newCell = cell.code
        .replace(/.*require\('\.\.\/Core'\)\s*;*\s*/ig, '')
        .replace(importerReg, '$1')
    exports.forEach(i => {
        let result = typeof i === 'string' ? allImports[i] : i.map(i => allImports[i]);
        if (typeof result === 'undefined' || result.length === 0) {
            throw new Error('Replacement not found ' + i);
        }
        if(result.length > 1 || (i.includes('.ipynb') && !i.includes('['))) {
            const imports = result
                .map(r => 'require(\'./' + (r.name = niceName(r)) + '\')')
                .join(', ');
            newCell = newCell.replace(
                new RegExp('((importer\\.)*import(Notebook)*)\\([\\s\\S]*?' + escapeRegExp(typeof i === 'string' ? i : i[0]) + '[\\s\\S]*?\\)', 'ig'),
                    `[${imports}].reduce((acc, r, i) => {
    if(typeof r === 'function') {
        acc[r.name] = r;
    } else if (r && typeof r[Object.keys(r)[0]] === 'function') {
        acc[Object.keys(r)[0]] = r[Object.keys(r)[0]];
        acc[r[Object.keys(r)[0]].name] = r[Object.keys(r)[0]];
    }
    acc[i] = r;
    return acc;
}, {})`.replace(/\n/igm, ''));
            return;
        } else {
            result = result[0] || result;
            newCell = newCell.replace(
                new RegExp('((importer\\.)*import(Notebook)*)\\([^\\)]*?[\'"]'
                           + escapeRegExp(typeof i === 'string' ? i : i[0])
                           + '[\'"][^\\)]*?\\)', 'ig'),
                    'require(\'./' + (result.name = niceName(result)) + '\')');
        }
    });
    return addImports(newCell, allImports);
}
module.exports = replaceImports;


export deploy notebook coordinator?

TODO: automatically find missing imports from distrib/Github/3rd party sources in any language?



In [ ]:
// assuming you've already run `aws configure`
var importer = require('../Core');
var {
    execCmd,
    delint,
    getImportsRecursively,
    replaceImports,
    getPermissions,
    injectImports,
    projectRelatives,
    outputExport
} = importer.import([
    'spawn child process',
    'delinting notebooks',
    'get imports recursively',
    'replace notebook imports',
    'rpc permissions',
    'inject cells notebooks webpack',
    'fix project paths',
    'output exported code'
]);

// TODO: exclude Core, defaults to false
function exportAndDeploy(notebook, projectOutput, matchOutput) {
    // these are the top level cells starting the import tree
    if(typeof notebook === 'string') {
        notebook = [notebook];
    }
    let entryCells = importer.interpret(notebook);
    if(!notebook || typeof entryCells.code === 'undefined' && typeof entryCells[0] === 'undefined') {
        throw new Error('No notebook found!');
    }
    
    // emit the file in every location request
    if(!projectOutput) {
        projectOutput = path.resolve(__dirname);
    }
    if(!matchOutput) {
        matchOutput = {};
    }
    //if(typeof matchOutput['**/*'] === 'undefined') {
    //    matchOutput["**/*"] = path.relative(projectOutput, path.resolve(path.join(__dirname, "../.output"))) + '/';
    //}
    
    const interpretedImports = {};
    const imports = getImportsRecursively(interpretedImports, entryCells, notebook);
    // only include core cache files if interpret notebook is used, like in RPC handlers
    if(Object.keys(interpretedImports).filter(k => k.includes('interpret notebook')).length > 0) {
        Object.assign(interpretedImports, injectImports(imports, interpretedImports));
    }

    var core = [];
    Object.keys(imports)
        .filter(i => interpretedImports[i].questions.length > 0
               || interpretedImports[i].questions[0].trim() != '')
        .forEach((e, i) => {
            // Add a code block header
            const exportedCode = replaceImports(imports[e], interpretedImports[e], interpretedImports);
            outputExport(exportedCode, interpretedImports[e], projectOutput, matchOutput)
        });
    
    // TODO: check index.js in output directory for missing files?
    // TODO: output packed cells and cache
    // TODO: zip and upload to AWS
    return projectRelatives(projectOutput);
    //return delint(PROJECT_PATH)
}

module.exports = exportAndDeploy;


test export code?

In [ ]:

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
    var project = PROFILE_PATH + '/Documents/window tracker';

    $$.async();
    exportAndDeploy('windows.ipynb', project, {
        '**/*.cs': '.'
    })
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


authoring header template?


In [ ]:

function authorTemplate(markdown, exportedCode) {
    return `
/**
 * Written by Brian Cullinan, exported using magic.
 * Copyright (c) ${(new Date()).getFullYear()} by Brian Cullinan, All rights reserved.
 *
${((markdown || '') + '').split('\n').map(l => ' * ' + l).join('\n')}
 *
 **/

` + exportedCode;
}

module.exports = authorTemplate;

output exported code?

In [ ]:
var fs = require('fs');
var os = require('os');
var path = require('path');
var minimatch = require('minimatch');
var importer = require('../Core')
var mkdirpSync = importer.import('mkdirp');
var authorTemplate = importer.import('authoring header template');

function outputExport(exportedCode, cell, projectOutput, matchOutput) {
    // some special exceptions with file-naming
    if(cell.name === '.js') {
        throw new Error('no filename! ' + JSON.stringify(cell));
    }
    if(cell.name.toLowerCase() === 'readme.js') {
        exportedCode = cell.markdown;
        cell.name = 'readme.md';
    }
    if(cell.name.toLowerCase() === 'package.js'
       || cell.name.toLowerCase() === 'package-json.js') {
        cell.name = 'package.json';
    }

    Object.keys(matchOutput).forEach(match => {
        if(minimatch(cell.name, match)) {
            // create directory if needed
            // TODO: move this to filesystem utility?
            const dir = matchOutput[match][matchOutput[match].length-1] === '/'
                ? matchOutput[match]
                : (fs.existsSync(matchOutput[match]) && fs.lstatSync(matchOutput[match]).isDirectory()
                   ? matchOutput[match]
                   : path.dirname(matchOutput[match]));
            if(!fs.existsSync(path.join(projectOutput, dir))) {
                mkdirpSync(path.resolve(path.join(projectOutput, dir)));
            }
            let filename = path.join(projectOutput,
                                     dir,
                                     path.basename(matchOutput[match]) === path.basename(dir)
                                     ? cell.name
                                     : path.basename(matchOutput[match]));
            console.log('emitting ' + filename);
            if(path.extname(filename) == '.js' || path.extname(filename) == '.cs') {
                // TODO: move this heading authoring template to utility function
                exportedCode = authorTemplate(cell.markdown, exportedCode)
            }
            fs.writeFileSync(filename, exportedCode);
        }
    });
}

module.exports = outputExport;


inject cells in to notebooks when built with webpack?


In [ ]:
var importer = require('../Core');
var path = require('path');

function camelCase(description) {
    return description.split(/\s+/ig).map((d, i) => i > 0 
                                          ? (d[0].toUpperCase() + d.substr(1)) 
                                          : d).join('')
}

// basic requirements for injecting different code into importer
var counter = 0;
function templateInject(description, code) {
    result = {};
    var name = camelCase(description);
    result[`${description}.ipynb[${counter}]`] = {
        markdown: `How to ${description}?`.split(/\n/ig).map(c => c + '\n'),
        language: 'javascript',
        cell_type: 'code',
        questions: [description],
        code: `
function ${name}() {
    return ${code};
}
module.exports = ${name};
`,
        filename: `../Utilities/notebook.ipynb/${description}[${counter}]`
    }
    counter++;
    return result;
}

// inject cellCache and cellIds loaded in to the interpreter so notebooks don't need to be uploaded, and webpack can tree-shake on functions already packed
// TODO: this should be a test in the Core notebook
function injectImports(imports, interpretedImports) {    
    const requires = [];
    const cells = Object.keys(interpretedImports).reduce((acc, k) => {
        const book = typeof interpretedImports[k].length === 'undefined'
            ? [interpretedImports[k]]
            : interpretedImports[k];
        if(typeof acc[book[0].filename] === 'undefined') {
            acc[book[0].filename] = [];
            const notebook = importer.interpret(path.basename(book[0].filename));
            notebook.forEach(cell => {
                // reference the function directly because it will be replaced by a file reference
                if(typeof imports[cell.id] !== 'undefined') {
                    requires.push(JSON.stringify(cell.id) + ': (ctx) => importer.import(' + JSON.stringify(cell.id) + ')');
                }
                cell.markdown.forEach(m => {
                    acc[cell.filename].push({
                        source: m.split(/\n/ig).map(l => l + '\n'),
                        language: cell.language,
                        cell_type: 'markdown'
                    });
                });
                acc[cell.filename].push({
                    source: cell.code.split(/\n/ig).map(l => l + '\n'),
                    language: cell.language,
                    cell_type: 'code'
                });
            });
        }
        return acc;
    }, {});
    
    const all = getPermissions();
    const permissions = Object.keys(all)
        .filter(k => typeof imports[all[k][1]] !== 'undefined')
        .reduce((acc, k) => (acc[k] = all[k], acc), {});

    imports['get injected.ipynb[0]'] = Object.keys(imports)
        .filter(k => !k.includes('get injected'));
    imports['get all cached.ipynb[1]'] = [];
    imports['get cached permissions.ipynb[2]'] = [];
    
    var injections = Object.assign({},
        templateInject('get injected', `{${requires.join(',\n        ')}}`),
        templateInject('get all cached', `${JSON.stringify(cells, null, 4)}`),
        templateInject('get cached permissions', `${JSON.stringify(permissions)}`));
    return injections;
}
module.exports = injectImports;
